Integrating an agent to dynamically retrieve and display model specifications from various AI providers' API documentation is an excellent approach to ensure up-to-date and comprehensive information. Here's how you can implement this:

**1. Define the Agent's Responsibilities:**

- **Fetch Model Lists:** Retrieve the current list of available models from each provider's API.

- **Gather Model Specifications:** For each model, extract relevant details such as context window size, input/output token limits, supported modalities, and any other pertinent information.

- **Update Local Database:** Maintain a local repository of these specifications to facilitate quick access and reduce redundant API calls.

**2. Implementing the Agent:**

You can create a Python script or module that performs the following tasks:

- **API Endpoints:** Identify and utilize the appropriate API endpoints or documentation URLs for each provider. For example:

  - **OpenAI:** Use the [Models API](https://platform.openai.com/docs/models) to list available models and their details.

  - **Anthropic:** Refer to the [Models List API](https://docs.anthropic.com/en/api/models-list) for information on Claude models.

  - **Google:** Access the [Vertex AI Model Garden](https://cloud.google.com/vertex-ai/docs/model-garden/overview) for details on available models.

- **Data Extraction:** Parse the JSON responses or HTML content to extract model specifications.

- **Database Update:** Store the extracted information in a structured format, such as a JSON or SQLite database, for easy retrieval.

**3. Scheduling Regular Updates:**

To ensure the information remains current, schedule the agent to run at regular intervals (e.g., daily or weekly) using a task scheduler like `cron` (Linux/macOS) or Task Scheduler (Windows).

**4. Integrating with Your Application:**

Modify your Streamlit application to query this local database when displaying model information. This approach ensures that users always see the most recent data without incurring the latency of real-time API calls.

**5. Handling API Changes:**

Implement error handling and logging within the agent to detect and alert you to any changes in the API structures or endpoints, allowing for prompt updates to the agent's code.

**6. Providing Fallback Information:**

In cases where API access is limited or unavailable, consider maintaining a fallback dataset with basic model information to ensure continuous functionality.

By implementing such an agent, you can automate the process of keeping your application's model specifications up-to-date, providing users with accurate and timely information. 

In [1]:
#!pip install requests
#!pip install openai
#!pip install anthropic
#!pip install google-genai


In [7]:
import os
import requests
import json
from dotenv import load_dotenv
from google import genai
from google.genai import types

# Load environment variables from .env
load_dotenv()

def get_env_var(var: str):
    """Retrieve an environment variable; raises error if not found."""
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Ensure it is set in your .env file.")
    return value

# Retrieve API keys from environment variables
openai_api_key     = get_env_var("OPENAI_API_COURSE_KEY")
anthropic_api_key  = get_env_var("ANTHROPIC_API_KEY")
google_api_key     = get_env_var("GOOGLE_API_KEY")
xai_api_key        = get_env_var("XAI_API_KEY")

def get_openai_models(api_key):
    url = "https://api.openai.com/v1/models"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            # Return only models starting with "gpt-"
            return [m["id"] for m in response.json().get("data", []) if m["id"].startswith("gpt-")]
        else:
            print("Failed to retrieve OpenAI models:", response.status_code, response.text)
            return []
    except Exception as e:
        print("Error retrieving OpenAI models:", e)
        return []

def get_anthropic_models(api_key):
    url = "https://api.anthropic.com/v1/models"
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",  # Ensure this version matches your API documentation
        "Content-Type": "application/json"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            return [m["id"] for m in response.json().get("data", []) if m["id"].startswith("claude")]
        else:
            print("Failed to retrieve Anthropic models:", response.status_code, response.text)
            return []
    except Exception as e:
        print("Error retrieving Anthropic models:", e)
        return []

def get_google_models(api_key):
    try:
        client = genai.Client(api_key=api_key)
        # Return model names (adjust if more details are needed)
        return [model.name for model in client.models.list()]
    except Exception as e:
        print("Failed to retrieve Google models:", e)
        return []

def get_grok_models(api_key):
    url = "https://api.x.ai/v1/models"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        models_data = response.json().get("data", [])
        return [model["id"] for model in models_data if "id" in model]
    except Exception as e:
        print("Failed to retrieve Grok models:", e)
        return []

# Fetch available models from each provider
openai_models    = get_openai_models(openai_api_key)
anthropic_models = get_anthropic_models(anthropic_api_key)
google_models    = get_google_models(google_api_key)
grok_models      = get_grok_models(xai_api_key)

# Assuming 'all_models' is your dictionary containing the model data:
all_models = {
    "openai": openai_models,
    "anthropic": anthropic_models,
    "google": google_models,
    "grok": grok_models
}

def count_total_models(models_dict):
    """Count the total number of models in the dictionary."""
    return sum(len(models) for models in models_dict.values())

# Count models and add the total to the dictionary
total_models = count_total_models(all_models)
all_models["total_models"] = total_models

# Save the updated dictionary to models_list.json
with open('models_list.json', 'w') as f:
    json.dump(all_models, f, indent=4)

print("Models list saved to models_list.json with a total count of:", total_models)


Models list saved to models_list.json with a total count of: 93


In [8]:
# Get the first model from the OpenAI models list
if openai_models:
    first_model_id = openai_models[0]
    print("Querying details for model:", first_model_id)

    # Build the URL for the model details endpoint
    model_url = f"https://api.openai.com/v1/models/{first_model_id}"
    headers = {"Authorization": f"Bearer {openai_api_key}", "Content-Type": "application/json"}
    
    # Query the model details
    response = requests.get(model_url, headers=headers, timeout=10)
    if response.ok:
        model_details = response.json()
        print("Model details:")
        print(json.dumps(model_details, indent=4))
    else:
        print(f"Failed to retrieve model details: {response.status_code} {response.text}")
else:
    print("No OpenAI models found.")


Querying details for model: gpt-4o-mini-transcribe
Model details:
{
    "id": "gpt-4o-mini-transcribe",
    "object": "model",
    "created": 1742068596,
    "owned_by": "system"
}


In [10]:
import os
import requests
import json
from dotenv import load_dotenv
from google import genai

# Load environment variables from .env
load_dotenv()

def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Ensure it is set in your .env file.")
    return value

# Retrieve API keys
openai_api_key     = get_env_var("OPENAI_API_COURSE_KEY")
anthropic_api_key  = get_env_var("ANTHROPIC_API_KEY")
google_api_key     = get_env_var("GOOGLE_API_KEY")
xai_api_key        = get_env_var("XAI_API_KEY")

# For Anthropic: Query details for the first Anthropic model in our list
def query_anthropic_model_details(api_key, model_id):
    url = f"https://api.anthropic.com/v1/models/{model_id}"
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "Content-Type": "application/json"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            return response.json()
        else:
            print(f"Failed to retrieve Anthropic model details: {response.status_code} {response.text}")
            return None
    except Exception as e:
        print("Error retrieving Anthropic model details:", e)
        return None

# For Google: Try to get details using the GenAI client.
# Note: The GenAI SDK might only return model names, so detailed metadata may not be available.
def query_google_model_details(api_key, model_name):
    try:
        client = genai.Client(api_key=api_key)
        # The GenAI client may not expose a dedicated "get details" method.
        # Here we simply return the model object from the list that matches the name.
        models = client.models.list()
        for model in models:
            if model.name == model_name:
                # Print available attributes; the SDK might not provide much detail.
                return {"name": model.name, "description": getattr(model, "description", "No description provided")}
        print("Model not found in Google models list.")
        return None
    except Exception as e:
        print("Error retrieving Google model details:", e)
        return None

# For Grok: Query details for a given Grok model.
def query_grok_model_details(api_key, model_id):
    url = f"https://api.x.ai/v1/models/{model_id}"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            return response.json()
        else:
            print(f"Failed to retrieve Grok model details: {response.status_code} {response.text}")
            return None
    except Exception as e:
        print("Error retrieving Grok model details:", e)
        return None

# Example lists (normally these come from your listing functions)
# For this example, assume we've already retrieved the lists:
anthropic_models = ["claude-3-7-sonnet-20250219", "claude-3-5-sonnet-20241022", "claude-3-5-haiku-20241022"]
google_models    = ["models/chat-bison-001", "models/text-bison-001"]  # using the model names as returned
grok_models      = ["grok-2-1212", "grok-2-vision-1212"]

# Query details for the first model from each provider
print("=== Anthropic Model Details ===")
if anthropic_models:
    anthro_details = query_anthropic_model_details(anthropic_api_key, anthropic_models[0])
    print(json.dumps(anthro_details, indent=4))
else:
    print("No Anthropic models found.")

print("\n=== Google Model Details ===")
if google_models:
    google_details = query_google_model_details(google_api_key, google_models[0])
    print(json.dumps(google_details, indent=4))
else:
    print("No Google models found.")

print("\n=== Grok Model Details ===")
if grok_models:
    grok_details = query_grok_model_details(xai_api_key, grok_models[0])
    print(json.dumps(grok_details, indent=4))
else:
    print("No Grok models found.")


=== Anthropic Model Details ===
{
    "type": "model",
    "id": "claude-3-7-sonnet-20250219",
    "display_name": "Claude 3.7 Sonnet",
    "created_at": "2025-02-24T00:00:00Z"
}

=== Google Model Details ===
{
    "name": "models/chat-bison-001",
    "description": "A legacy text-only model optimized for chat conversations"
}

=== Grok Model Details ===
{
    "id": "grok-2-1212",
    "created": 1737331200,
    "object": "model",
    "owned_by": "xai"
}


Based on the responses we received:

**Anthropic returns:**
- **type:** The kind of object (in this case, a model)
- **id:** The unique identifier for the model
- **display_name:** A human-friendly name (e.g., "Claude 3.7 Sonnet")
- **created_at:** A timestamp indicating when the model was created

**Google (via the GenAI SDK) returns:**
- **name:** The model’s identifier (including a path-like string, e.g., "models/chat-bison-001")
- **description:** A brief description of the model (e.g., "A legacy text-only model optimized for chat conversations")

**Grok (xAI) returns:**
- **id:** The unique identifier for the model
- **created:** A creation timestamp (in Unix time)
- **object:** Typically indicating the object type (here, "model")
- **owned_by:** The owner (e.g., "xai")

**OpenAI returns:**
- **id:** The unique identifier for the model (e.g., "gpt-4o-mini-transcribe")
- **object:** The object type (typically "model")
- **created:** The creation timestamp (in Unix time)
- **owned_by:** The owner (which can be "system" or "openai")

This consolidated view helps you compare the metadata available from each provider and determine what additional details you might need to supplement through internal configuration or documentation.

**What Else Can Be Retrieved Automatically?**

The information you can automatically retrieve depends on each provider’s API design. Here are some possibilities:

1. **Basic Metadata:**  
   Most endpoints return basic metadata like model IDs, creation timestamps, and sometimes a human-friendly name or description.

2. **Permissions and Access Controls:**  
   Some APIs may include fields indicating who can access or use a particular model (e.g., `"owned_by"` or `"permissions"` lists).

3. **Status Information:**  
   In some cases, the API might include information on the model’s current status or whether it’s in preview, beta, or fully released.

4. **Versioning:**  
   The response might contain version-related details (e.g., in the model ID or a dedicated version field).

**What Isn’t Typically Retrieved Automatically?**

Many technical details like:
  
- **Parameter Count or Model Size**
- **Maximum Context Window (Input/Output Token Limits)**
- **Supported Modalities (e.g., text, image, audio)**
- **Latency or Performance Metrics**

...are usually not provided in the basic listing endpoints. This extra information is often found in provider documentation or requires separate endpoints or manual configuration.

**Next Steps:**

If you need more detailed technical specifications (like context window size or parameter counts), you may need to:
  
- **Reference the API Documentation:** Supplement the API response with data from official documentation.
- **Create an Internal Repository:** Manually compile and maintain these details in your application (or use a scheduled process that parses documentation pages if available).

This approach lets you combine automatically retrieved metadata with additional technical details necessary for your application's logic.

Based on OpenAI's official API documentation and the response we received, here's what we can automatically retrieve from the `/v1/models` endpoint for an OpenAI model:

- **id:** The unique identifier for the model (e.g., `"gpt-4o-mini-transcribe"`).
- **object:** The type of the returned object (typically `"model"`).
- **created:** A Unix timestamp indicating when the model was created.
- **owned_by:** Information about who owns the model (this might be `"system"` for system-managed models or `"openai"` for those managed by OpenAI).

### What Else Might Be Available?

When analyzing OpenAI's API information on their documentation page, you'll notice that the models endpoint intentionally returns only minimal metadata. Additional technical details like:

- **Maximum Context Length (Input/Output Token Limits)**
- **Parameter Count or Model Size**
- **Supported Modalities (e.g., text, image, audio)**
- **Performance Metrics or Latency Data**

...are not included in the automatic API response. These details are typically found in the official documentation and not returned via the API itself.

### Why Is This the Case?

OpenAI’s design for the `/v1/models` endpoint is to provide a list of models and basic metadata. The assumption is that any deeper technical details (e.g., context window or parameter count) will be managed internally or referenced from their documentation. This means that if you need to display or utilize such details in your application, you’ll likely have to supplement the API response with internal configuration (such as your `MODEL_TOKEN_LIMITS` dictionary) or a manual update based on the latest documentation.

### Next Steps

1. **Review the Documentation:**  
   Check OpenAI's API reference pages (such as [Models API](https://platform.openai.com/docs/api-reference/models)) for any additional details that might be documented but not exposed via the API.

2. **Supplement Data:**  
   If you need details like context window size or parameter count, consider maintaining an internal reference table (like your existing `MODEL_TOKEN_LIMITS` dictionary) or manually parsing their documentation.

3. **Consider Future Endpoints:**  
   Occasionally, APIs evolve. Keep an eye on OpenAI's announcements or API changelogs in case they decide to expose more metadata in the future.

In summary, the automatically retrievable information from OpenAI's models endpoint is limited to basic metadata, and additional details must be supplemented by manual configuration or referenced from external documentation.

In [12]:
import os
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables.")
    return value

openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")

# Query details for a specific model
model_id = "gpt-4o-2024-05-13"
model_url = f"https://api.openai.com/v1/models/{model_id}"
headers = {"Authorization": f"Bearer {openai_api_key}", "Content-Type": "application/json"}

response = requests.get(model_url, headers=headers, timeout=10)

if response.ok:
    model_details = response.json()
    print("Model details:")
    print(json.dumps(model_details, indent=4))
    
    # Extract and print relevant headers for debugging and rate limiting
    response_headers = response.headers
    debug_info = {
        "openai-organization": response_headers.get("openai-organization"),
        "openai-processing-ms": response_headers.get("openai-processing-ms"),
        "openai-version": response_headers.get("openai-version"),
        "x-request-id": response_headers.get("x-request-id"),
        "x-ratelimit-limit-requests": response_headers.get("x-ratelimit-limit-requests"),
        "x-ratelimit-limit-tokens": response_headers.get("x-ratelimit-limit-tokens"),
        "x-ratelimit-remaining-requests": response_headers.get("x-ratelimit-remaining-requests"),
        "x-ratelimit-remaining-tokens": response_headers.get("x-ratelimit-remaining-tokens"),
        "x-ratelimit-reset-requests": response_headers.get("x-ratelimit-reset-requests"),
        "x-ratelimit-reset-tokens": response_headers.get("x-ratelimit-reset-tokens"),
    }
    print("\nDebug/Rate Limiting Headers:")
    print(json.dumps(debug_info, indent=4))
else:
    print(f"Failed to retrieve model details: {response.status_code} {response.text}")


Model details:
{
    "id": "gpt-4o-2024-05-13",
    "object": "model",
    "created": 1715368132,
    "owned_by": "system"
}

Debug/Rate Limiting Headers:
{
    "openai-organization": null,
    "openai-processing-ms": "105",
    "openai-version": "2020-10-01",
    "x-request-id": "2ac1c625553f3c3d339759af2ab56f08",
    "x-ratelimit-limit-requests": null,
    "x-ratelimit-limit-tokens": null,
    "x-ratelimit-remaining-requests": null,
    "x-ratelimit-remaining-tokens": null,
    "x-ratelimit-reset-requests": null,
    "x-ratelimit-reset-tokens": null
}


The response for the model `"gpt-4o-2024-05-13"` is very similar to the previous one in structure. Here’s what we can glean from it:

- **Model Details:**
  - **id:** `"gpt-4o-2024-05-13"`  
    This uniquely identifies the model.
  - **object:** `"model"`  
    Indicates the type of the object returned.
  - **created:** `1715368132`  
    A Unix timestamp indicating when the model was created.
  - **owned_by:** `"system"`  
    Indicates that this model is managed by the system.

- **Debug/Rate Limiting Headers:**
  - **openai-processing-ms:** `"105"`  
    Indicates that the API took 105 milliseconds to process the request.
  - **openai-version:** `"2020-10-01"`  
    Shows the API version used.
  - **x-request-id:** `"2ac1c625553f3c3d339759af2ab56f08"`  
    Provides a unique identifier for this API request, useful for debugging.
  - **Rate Limiting Headers:**  
    All related headers (like limits and remaining counts) are null, which is expected for this endpoint.

### Why This Information Might Be Useful

- **Request Identification & Debugging:**  
  The `x-request-id` can help you trace and debug issues with API requests, particularly in production or when working with support teams.

- **Performance Insights:**  
  The `openai-processing-ms` value gives you an idea of the latency for the API call, which can be valuable for performance monitoring.

- **API Versioning:**  
  Knowing the `openai-version` ensures you are aware of which version of the API you are interacting with, which is crucial for compatibility.

### Summary

While the model details endpoint provides only minimal metadata, the accompanying HTTP headers give you valuable information for debugging and monitoring purposes. For deeper technical details (like maximum context window or model size), you would typically rely on OpenAI's documentation or internal configurations.

If you need to automatically enrich your model information, you might combine these API responses with manually maintained data (like your `MODEL_TOKEN_LIMITS` dictionary) or use separate endpoints if available.

In [15]:
import os
import requests
import json
from dotenv import load_dotenv
from google import genai
from google.genai import types

# Load environment variables from .env
load_dotenv()

def get_env_var(var: str):
    """Retrieve an environment variable; raises error if not found."""
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Ensure it is set in your .env file.")
    return value

# Retrieve API keys from environment variables
openai_api_key     = get_env_var("OPENAI_API_COURSE_KEY")
anthropic_api_key  = get_env_var("ANTHROPIC_API_KEY")
google_api_key     = get_env_var("GOOGLE_API_KEY")
xai_api_key        = get_env_var("XAI_API_KEY")

# Functions to dynamically list models from each provider
def get_openai_models(api_key):
    url = "https://api.openai.com/v1/models"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            return [m["id"] for m in response.json().get("data", []) if m["id"].startswith("gpt-")]
        else:
            print("Failed to retrieve OpenAI models:", response.status_code, response.text)
            return []
    except Exception as e:
        print("Error retrieving OpenAI models:", e)
        return []

def get_anthropic_models(api_key):
    url = "https://api.anthropic.com/v1/models"
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "Content-Type": "application/json"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.ok:
            return [m["id"] for m in response.json().get("data", []) if m["id"].startswith("claude")]
        else:
            print("Failed to retrieve Anthropic models:", response.status_code, response.text)
            return []
    except Exception as e:
        print("Error retrieving Anthropic models:", e)
        return []

def get_google_models(api_key):
    try:
        client = genai.Client(api_key=api_key)
        return [model.name for model in client.models.list()]
    except Exception as e:
        print("Failed to retrieve Google models:", e)
        return []

def get_grok_models(api_key):
    url = "https://api.x.ai/v1/models"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        models_data = response.json().get("data", [])
        return [model["id"] for model in models_data if "id" in model]
    except Exception as e:
        print("Failed to retrieve Grok models:", e)
        return []

# Load dynamic models from all providers
openai_models    = get_openai_models(openai_api_key)
anthropic_models = get_anthropic_models(anthropic_api_key)
google_models    = get_google_models(google_api_key)
grok_models      = get_grok_models(xai_api_key)

dynamic_models = {
    "openai": openai_models,
    "anthropic": anthropic_models,
    "google": google_models,
    "grok": grok_models
}

# -----------------------------------------------------------------------------
# Load additional metadata from a JSON file (if available)
# -----------------------------------------------------------------------------
def load_models_metadata(filename="models_metadata.json"):
    if os.path.exists(filename):
        try:
            with open(filename, "r") as f:
                return json.load(f)
        except Exception as e:
            print(f"Error loading metadata from {filename}: {e}")
            return {}
    else:
        print(f"{filename} not found. No additional metadata will be added.")
        return {}

models_metadata = load_models_metadata("models_metadata.json")

# -----------------------------------------------------------------------------
# Enrich dynamic models with metadata (if available)
# -----------------------------------------------------------------------------
def enrich_models(dynamic_models, metadata):
    enriched = {}
    for provider, models in dynamic_models.items():
        enriched[provider] = []
        for model in models:
            model_metadata = metadata.get(model, {})  # Get metadata if exists; else empty dict
            enriched[provider].append({
                "model_id": model,
                "metadata": model_metadata
            })
    return enriched

enriched_models = enrich_models(dynamic_models, models_metadata)

# -----------------------------------------------------------------------------
# Count total number of models and add to the dictionary
# -----------------------------------------------------------------------------
def count_total_models(models_dict):
    return sum(len(models) for models in models_dict.values())

total_models = count_total_models(dynamic_models)
enriched_models["total_models"] = total_models

# -----------------------------------------------------------------------------
# Save enriched models list to models_list.json
# -----------------------------------------------------------------------------
with open('models_list.json', 'w') as f:
    json.dump(enriched_models, f, indent=4)

print("Models list saved to models_list.json with a total count of:", total_models)


Error loading metadata from models_metadata.json: Expecting property name enclosed in double quotes: line 20 column 5 (char 684)
Models list saved to models_list.json with a total count of: 93


In [16]:
import requests
from bs4 import BeautifulSoup
import re
import json

# URL of the OpenAI models information page (example URL; update as needed)
url = "https://platform.openai.com/docs/api-reference/models"

# Fetch the page
response = requests.get(url)
if response.status_code != 200:
    print("Error fetching page:", response.status_code)
    exit()

html = response.text
soup = BeautifulSoup(html, "html.parser")

# Initialize a dictionary to store metadata for each model.
# This is a draft approach; you will need to adjust the selectors based on the actual HTML.
models_metadata = {}

# Suppose that each model is described within a section with a specific CSS class.
# (This is a placeholder selector; update according to the actual page.)
model_sections = soup.find_all("div", class_="model-section")

for section in model_sections:
    # Attempt to extract the model ID/name from a heading.
    model_id_elem = section.find("h2")
    if not model_id_elem:
        continue
    model_id = model_id_elem.text.strip()
    
    # Extract a description. For example, assume there's a <p> with class "description".
    description_elem = section.find("p", class_="description")
    description = description_elem.text.strip() if description_elem else "TBD"
    
    # Get the full text of the section to search for token limits
    section_text = section.get_text()
    
    # Use regular expressions to look for token limits; adjust patterns as needed.
    max_input_match = re.search(r"Max Input Tokens:\s*(\d+)", section_text)
    max_output_match = re.search(r"Max Output Tokens:\s*(\d+)", section_text)
    context_window_match = re.search(r"Context Window:\s*(\d+)", section_text)
    
    max_input_tokens = int(max_input_match.group(1)) if max_input_match else "TBD"
    max_output_tokens = int(max_output_match.group(1)) if max_output_match else "TBD"
    context_window = int(context_window_match.group(1)) if context_window_match else "TBD"
    
    models_metadata[model_id] = {
        "max_input_tokens": max_input_tokens,
        "max_output_tokens": max_output_tokens,
        "context_window": context_window,
        "description": description
    }

# For demonstration purposes, add a sample entry if no sections were found:
if not models_metadata:
    models_metadata = {
        "gpt-4": {
            "max_input_tokens": 8192,
            "max_output_tokens": 8192,
            "context_window": 8192,
            "description": "GPT-4: A large multimodal model capable of processing text and images. Input: text, images; Output: text."
        }
    }

# Save the metadata to models_metadata_scraped.json
with open("models_metadata.json", "w") as f:
    json.dump(models_metadata, f, indent=4)

print("Scraped models metadata saved to models_metadata.json")


Error fetching page: 403
Scraped models metadata saved to models_metadata.json


In [3]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 63.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# URL for the OpenAI models page
OVERVIEW_URL = "https://platform.openai.com/docs/models"

def fetch_dynamic_page(url):
    # Configure Selenium to run in headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    # Set up the webdriver (make sure chromedriver is installed and in PATH)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    # Wait for the dynamic content to load (adjust sleep as needed)
    time.sleep(5)
    html = driver.page_source
    driver.quit()
    return html

def parse_overview_page(html):
    """Parse the dynamically loaded overview page to extract model names and their URLs."""
    soup = BeautifulSoup(html, "html.parser")
    models = {}
    
    # Inspect the page source (using your browser's developer tools) to find
    # the appropriate selectors. For example, suppose model links are in <a> tags
    # with a class "model-link":
    for a in soup.find_all("a", class_="model-link", href=True):
        href = a["href"]
        model_id = href.split("/")[-1].strip()  # e.g., "gpt-4"
        # Construct the full URL for the model page
        model_url = "https://platform.openai.com" + href
        models[model_id] = model_url
    return models

def parse_model_page(html):
    """Parse an individual model page to extract metadata and pricing info."""
    soup = BeautifulSoup(html, "html.parser")
    metadata = {
        "max_input_tokens": "TBD",
        "max_output_tokens": "TBD",
        "context_window": "TBD",
        "description": "TBD",
        "pricing": "TBD"
    }
    
    # These selectors are examples. You will need to inspect the page and adjust:
    desc_elem = soup.find("div", class_="model-description")
    if desc_elem:
        metadata["description"] = desc_elem.get_text(separator=" ", strip=True)
    
    full_text = soup.get_text(separator=" ", strip=True)
    # Use regex to find token limits and pricing if available
    input_match = re.search(r"Max Input Tokens:\s*(\d+)", full_text)
    output_match = re.search(r"Max Output Tokens:\s*(\d+)", full_text)
    context_match = re.search(r"Context Window:\s*(\d+)", full_text)
    pricing_match = re.search(r"Pricing:\s*([\$\d\.]+)", full_text)
    
    if input_match:
        metadata["max_input_tokens"] = int(input_match.group(1))
    if output_match:
        metadata["max_output_tokens"] = int(output_match.group(1))
    if context_match:
        metadata["context_window"] = int(context_match.group(1))
    if pricing_match:
        metadata["pricing"] = pricing_match.group(1)
    
    return metadata

def scrape_all_models():
    """Scrape the OpenAI models overview page, then scrape each model page for metadata."""
    overview_html = fetch_dynamic_page(OVERVIEW_URL)
    if not overview_html:
        return {}
    
    models = parse_overview_page(overview_html)
    print(f"Found {len(models)} models on the overview page.")
    
    models_metadata = {}
    
    for model_id, model_url in models.items():
        print(f"Scraping metadata for {model_id} from {model_url}")
        model_html = fetch_dynamic_page(model_url)
        if model_html:
            metadata = parse_model_page(model_html)
        else:
            metadata = {
                "max_input_tokens": "TBD",
                "max_output_tokens": "TBD",
                "context_window": "TBD",
                "description": "TBD",
                "pricing": "TBD"
            }
        models_metadata[model_id] = metadata
    
    return models_metadata

def save_metadata(metadata, filename="models_metadata_scraped.json"):
    with open(filename, "w") as f:
        json.dump(metadata, f, indent=4)
    print(f"Models metadata saved to {filename}")

if __name__ == "__main__":
    metadata = scrape_all_models()
    save_metadata(metadata)


Found 0 models on the overview page.
Models metadata saved to models_metadata_scraped.json


In [2]:
import os
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# URL of the OpenAI models overview page
OVERVIEW_URL = "https://platform.openai.com/docs/models"

def setup_driver():
    """Set up Selenium Chrome driver in headless mode."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    # You may add further options if needed
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def fetch_dynamic_page(url, driver, wait_time=10):
    """Fetch a dynamically rendered page using Selenium and wait for model links."""
    driver.get(url)
    try:
        # Wait for at least one <a> tag containing '/docs/models/' in href
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/docs/models/']"))
        )
    except Exception as e:
        print("Timeout waiting for dynamic content:", e)
    # Give an extra second for safety
    time.sleep(1)
    return driver.page_source

def parse_overview_page(html):
    """Parse the overview page to extract model names and their URLs."""
    soup = BeautifulSoup(html, "html.parser")
    models = {}
    # Find all <a> tags whose href contains "/docs/models/"
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/docs/models/" in href:
            parts = href.split("/")
            if len(parts) > 2:
                model_id = parts[-1].strip()
                # Filter out empty strings and duplicates
                if model_id and model_id not in models:
                    model_url = "https://platform.openai.com" + href
                    models[model_id] = model_url
    return models

def parse_model_page(html):
    """Parse an individual model page to extract metadata and pricing info."""
    soup = BeautifulSoup(html, "html.parser")
    metadata = {
        "max_input_tokens": "TBD",
        "max_output_tokens": "TBD",
        "context_window": "TBD",
        "description": "TBD",
        "pricing": "TBD"
    }
    # Example: Try to extract description from a known container (adjust selector)
    desc_elem = soup.find("div", class_="model-description")
    if desc_elem:
        metadata["description"] = desc_elem.get_text(separator=" ", strip=True)
    
    full_text = soup.get_text(separator=" ", strip=True)
    # Use regex to extract token limits and pricing
    input_match = re.search(r"Max Input Tokens:\s*(\d+)", full_text)
    output_match = re.search(r"Max Output Tokens:\s*(\d+)", full_text)
    context_match = re.search(r"Context Window:\s*(\d+)", full_text)
    pricing_match = re.search(r"Pricing:\s*([\$\d\.]+)", full_text)
    
    if input_match:
        metadata["max_input_tokens"] = int(input_match.group(1))
    if output_match:
        metadata["max_output_tokens"] = int(output_match.group(1))
    if context_match:
        metadata["context_window"] = int(context_match.group(1))
    if pricing_match:
        metadata["pricing"] = pricing_match.group(1)
    
    return metadata

def scrape_all_models():
    """Scrape the OpenAI models overview page and then each model page for metadata."""
    driver = setup_driver()
    overview_html = fetch_dynamic_page(OVERVIEW_URL, driver)
    models = parse_overview_page(overview_html)
    print(f"Found {len(models)} models on the overview page.")
    
    models_metadata = {}
    for model_id, model_url in models.items():
        print(f"Scraping metadata for {model_id} from {model_url}")
        model_html = fetch_dynamic_page(model_url, driver)
        if model_html:
            metadata = parse_model_page(model_html)
        else:
            metadata = {
                "max_input_tokens": "TBD",
                "max_output_tokens": "TBD",
                "context_window": "TBD",
                "description": "TBD",
                "pricing": "TBD"
            }
        models_metadata[model_id] = metadata
    driver.quit()
    return models_metadata

def save_metadata(metadata, filename="models_metadata_scraped.json"):
    with open(filename, "w") as f:
        json.dump(metadata, f, indent=4)
    print(f"Models metadata saved to {filename}")

if __name__ == "__main__":
    metadata = scrape_all_models()
    save_metadata(metadata)


Timeout waiting for dynamic content: Message: 
Stacktrace:
0   chromedriver                        0x0000000103315818 chromedriver + 6105112
1   chromedriver                        0x000000010330d41a chromedriver + 6071322
2   chromedriver                        0x0000000102da8600 chromedriver + 415232
3   chromedriver                        0x0000000102dfa2c0 chromedriver + 750272
4   chromedriver                        0x0000000102dfa511 chromedriver + 750865
5   chromedriver                        0x0000000102e4a9c4 chromedriver + 1079748
6   chromedriver                        0x0000000102e2063d chromedriver + 906813
7   chromedriver                        0x0000000102e47c3d chromedriver + 1068093
8   chromedriver                        0x0000000102e203e3 chromedriver + 906211
9   chromedriver                        0x0000000102dec29a chromedriver + 692890
10  chromedriver                        0x0000000102ded3f1 chromedriver + 697329
11  chromedriver                        0x0000

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get("https://platform.openai.com/docs/models")
time.sleep(5)  # wait for dynamic content to load
page_source = driver.page_source
driver.quit()

with open("dynamic_page_source.html", "w", encoding="utf-8") as f:
    f.write(page_source)
print("Dynamic page source saved to dynamic_page_source.html")


Dynamic page source saved to dynamic_page_source.html


In [4]:
import requests

url = "https://platform.openai.com/docs/models"
response = requests.get(url)
if response.ok:
    with open("page_source.html", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("Page source saved to page_source.html")
else:
    print("Failed to fetch page:", response.status_code)


Failed to fetch page: 403


In [5]:
import os
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def setup_driver():
    """Set up Selenium Chrome driver in headless mode."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def fetch_dynamic_page(url, driver, wait_time=15):
    """Fetch a dynamically rendered page using Selenium."""
    driver.get(url)
    # Optionally scroll to bottom to force lazy-loading content
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(wait_time)
    return driver.page_source

def find_model_links(html):
    """
    Parse the overview page HTML and extract links that follow the pattern:
    "https://platform.openai.com/docs/models/<model_id>"
    """
    soup = BeautifulSoup(html, "html.parser")
    model_links = {}
    
    # Loop through all <a> tags that have an href
    for a in soup.find_all("a", href=True):
        href = a['href']
        # Check if the link points to a model page. It might be a relative URL.
        if href.startswith("/docs/models/") or href.startswith("https://platform.openai.com/docs/models/"):
            # Construct full URL if needed
            if href.startswith("/docs/models/"):
                full_url = "https://platform.openai.com" + href
            else:
                full_url = href
            # Extract model id from the URL (the last part)
            model_id = full_url.split("/")[-1].strip()
            if model_id and model_id not in model_links:
                model_links[model_id] = full_url
    return model_links

def main():
    overview_url = "https://platform.openai.com/docs/models"
    driver = setup_driver()
    html = fetch_dynamic_page(overview_url, driver, wait_time=15)
    driver.quit()
    
    model_links = find_model_links(html)
    print("Found models:")
    print(json.dumps(model_links, indent=4))
    
    # Save the model links to a JSON file
    with open("openai_model_links.json", "w") as f:
        json.dump(model_links, f, indent=4)
    print("Model links saved to openai_model_links.json")

if __name__ == "__main__":
    main()


Found models:
{}
Model links saved to openai_model_links.json


In [6]:
import os
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# URL of the OpenAI models overview page
OVERVIEW_URL = "https://platform.openai.com/docs/models"

def setup_driver():
    """Set up Selenium Chrome driver in headless mode."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def fetch_dynamic_page(url, driver, wait_time=15):
    """Fetch a dynamically rendered page using Selenium."""
    driver.get(url)
    # Scroll to bottom to force lazy-loading
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        # Wait for an <a> tag in the sidebar that starts with /docs/models/
        WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "nav a[href^='/docs/models/']"))
        )
    except Exception as e:
        print("Timeout waiting for dynamic content:", e)
    time.sleep(2)  # Extra wait time
    return driver.page_source

def parse_overview_page(html):
    """Parse the overview page to extract model links based on side navigation."""
    soup = BeautifulSoup(html, "html.parser")
    models = {}
    
    # First, try to find the navigation element that likely contains model links.
    nav = soup.find("nav")
    if nav:
        for a in nav.find_all("a", href=True):
            href = a["href"]
            if href.startswith("/docs/models/"):
                model_id = href.split("/")[-1].strip()
                if model_id:
                    full_url = "https://platform.openai.com" + href
                    models[model_id] = full_url
    else:
        # Fallback: search all <a> tags
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "/docs/models/" in href:
                model_id = href.split("/")[-1].strip()
                if model_id:
                    full_url = "https://platform.openai.com" + href if href.startswith("/") else href
                    models[model_id] = full_url
    return models

def parse_model_page(html):
    """Parse an individual model page to extract metadata and pricing info."""
    soup = BeautifulSoup(html, "html.parser")
    metadata = {
        "max_input_tokens": "TBD",
        "max_output_tokens": "TBD",
        "context_window": "TBD",
        "description": "TBD",
        "pricing": "TBD"
    }
    
    # Example: Extract description from a container with a specific class (adjust as needed)
    desc_elem = soup.find("div", class_="model-description")
    if desc_elem:
        metadata["description"] = desc_elem.get_text(separator=" ", strip=True)
    
    # Use regex to extract token limits and pricing from the full page text
    full_text = soup.get_text(separator=" ", strip=True)
    input_match = re.search(r"Max Input Tokens:\s*(\d+)", full_text)
    output_match = re.search(r"Max Output Tokens:\s*(\d+)", full_text)
    context_match = re.search(r"Context Window:\s*(\d+)", full_text)
    pricing_match = re.search(r"Pricing:\s*([\$\d\.]+)", full_text)
    
    if input_match:
        metadata["max_input_tokens"] = int(input_match.group(1))
    if output_match:
        metadata["max_output_tokens"] = int(output_match.group(1))
    if context_match:
        metadata["context_window"] = int(context_match.group(1))
    if pricing_match:
        metadata["pricing"] = pricing_match.group(1)
    
    return metadata

def scrape_all_models():
    """Scrape the OpenAI models overview page, then each model page for metadata."""
    driver = setup_driver()
    overview_html = fetch_dynamic_page(OVERVIEW_URL, driver, wait_time=15)
    models = parse_overview_page(overview_html)
    print(f"Found {len(models)} models on the overview page.")
    
    models_metadata = {}
    for model_id, model_url in models.items():
        print(f"Scraping metadata for {model_id} from {model_url}")
        model_html = fetch_dynamic_page(model_url, driver, wait_time=10)
        if model_html:
            metadata = parse_model_page(model_html)
        else:
            metadata = {
                "max_input_tokens": "TBD",
                "max_output_tokens": "TBD",
                "context_window": "TBD",
                "description": "TBD",
                "pricing": "TBD"
            }
        models_metadata[model_id] = metadata
    driver.quit()
    return models_metadata

def save_metadata(metadata, filename="models_metadata_scraped.json"):
    """Save the scraped metadata to a JSON file."""
    with open(filename, "w") as f:
        json.dump(metadata, f, indent=4)
    print(f"Models metadata saved to {filename}")

if __name__ == "__main__":
    metadata = scrape_all_models()
    save_metadata(metadata)


Timeout waiting for dynamic content: Message: 
Stacktrace:
0   chromedriver                        0x000000010f755818 chromedriver + 6105112
1   chromedriver                        0x000000010f74d41a chromedriver + 6071322
2   chromedriver                        0x000000010f1e8600 chromedriver + 415232
3   chromedriver                        0x000000010f23a2c0 chromedriver + 750272
4   chromedriver                        0x000000010f23a511 chromedriver + 750865
5   chromedriver                        0x000000010f28a9c4 chromedriver + 1079748
6   chromedriver                        0x000000010f26063d chromedriver + 906813
7   chromedriver                        0x000000010f287c3d chromedriver + 1068093
8   chromedriver                        0x000000010f2603e3 chromedriver + 906211
9   chromedriver                        0x000000010f22c29a chromedriver + 692890
10  chromedriver                        0x000000010f22d3f1 chromedriver + 697329
11  chromedriver                        0x0000

Featured models
https://platform.openai.com/docs/models/gpt-4.5-preview
https://platform.openai.com/docs/models/o3-mini
https://platform.openai.com/docs/models/gpt-4o

Reasoning models 
o-series models that excel at complex, multi-step tasks.
https://platform.openai.com/docs/models/o3-mini
https://platform.openai.com/docs/models/o1
https://platform.openai.com/docs/models/o1-mini
https://platform.openai.com/docs/models/o1-pro

Flagship chat models 
Our versatile, high-intelligence flagship models.
https://platform.openai.com/docs/models/gpt-4.5-preview
https://platform.openai.com/docs/models/gpt-4o-audio-preview
https://platform.openai.com/docs/models/gpt-4o
https://platform.openai.com/docs/models/chatgpt-4o-latest

Cost-optimized models 
Smaller, faster models that cost less to run.
https://platform.openai.com/docs/models/gpt-4o-mini
https://platform.openai.com/docs/models/gpt-4o-mini-audio-preview

Realtime models 
Models capable of realtime text and audio inputs and outputs.
https://platform.openai.com/docs/models/gpt-4o-realtime-preview
https://platform.openai.com/docs/models/gpt-4o-mini-realtime-preview

Older GPT models 
Supported older versions of our general purpose and chat models.
https://platform.openai.com/docs/models/gpt-4-turbo
https://platform.openai.com/docs/models/gpt-4
https://platform.openai.com/docs/models/gpt-3.5-turbo

DALL·E
Models that can generate and edit images, given a natural language prompt.
https://platform.openai.com/docs/models/dall-e-3
https://platform.openai.com/docs/models/dall-e-2

Text-to-speech
Models that can convert text into natural sounding spoken audio.
https://platform.openai.com/docs/models/gpt-4o-mini-tts
https://platform.openai.com/docs/models/tts-1-hd
https://platform.openai.com/docs/models/tts-1

Transcription
Model that can transcribe and translate audio into text.
https://platform.openai.com/docs/models/gpt-4o-transcribe
https://platform.openai.com/docs/models/whisper-1
https://platform.openai.com/docs/models/gpt-4o-mini-transcribe

Embeddings
A set of models that can convert text into vector representations.
https://platform.openai.com/docs/models/text-embedding-3-small
https://platform.openai.com/docs/models/text-embedding-ada-002
https://platform.openai.com/docs/models/text-embedding-3-large

Moderation
Fine-tuned models that detect whether input may be sensitive or unsafe.
https://platform.openai.com/docs/models/omni-moderation-latest
https://platform.openai.com/docs/models/text-moderation-latest

Tool-specific models
Models to support specific built-in tools.
https://platform.openai.com/docs/models/gpt-4o-search-preview
https://platform.openai.com/docs/models/gpt-4o-mini-search-preview
https://platform.openai.com/docs/models/computer-use-preview

GPT base models
Older models that aren't trained with instruction following.
https://platform.openai.com/docs/models/babbage-002
https://platform.openai.com/docs/models/davinci-002